# Model Fit in Linear Regression - Lab

## Introduction
In this lab, you'll learn how to evaluate your model results, and you'll learn methods to select the appropriate features using stepwise selection.

## Objectives
You will be able to:
* Analyze the results of regression and R-squared and adjusted-R-squared 
* Understand and apply forward and backward predictor selection

## The Boston Housing Data once more

We pre-processed the Boston Housing Data the same way we did before:

- We dropped "ZN" and "NOX" completely
- We categorized "RAD" in 3 bins and "TAX" in 4 bins
- We used min-max-scaling on "B", "CRIM" and "DIS" (and logtransformed all of them first, except "B")
- We used standardization on "AGE", "INDUS", "LSTAT" and "PTRATIO" (and logtransformed all of them first, except for "AGE") 

In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_boston
boston = load_boston()

boston_features = pd.DataFrame(boston.data, columns = boston.feature_names)
boston_features = boston_features.drop(["NOX","ZN"],axis=1)

# first, create bins for based on the values observed. 3 values will result in 2 bins
bins = [0,6,  24]
bins_rad = pd.cut(boston_features['RAD'], bins)
bins_rad = bins_rad.cat.as_unordered()

# first, create bins for based on the values observed. 4 values will result in 3 bins
bins = [0, 270, 360, 712]
bins_tax = pd.cut(boston_features['TAX'], bins)
bins_tax = bins_tax.cat.as_unordered()

tax_dummy = pd.get_dummies(bins_tax, prefix="TAX")
rad_dummy = pd.get_dummies(bins_rad, prefix="RAD")
boston_features = boston_features.drop(["RAD","TAX"], axis=1)
boston_features = pd.concat([boston_features, rad_dummy, tax_dummy], axis=1)

age = boston_features["AGE"]
b = boston_features["B"]
logcrim = np.log(boston_features["CRIM"])
logdis = np.log(boston_features["DIS"])
logindus = np.log(boston_features["INDUS"])
loglstat = np.log(boston_features["LSTAT"])
logptratio = np.log(boston_features["PTRATIO"])

# minmax scaling
boston_features["B"] = (b-min(b))/(max(b)-min(b))
boston_features["CRIM"] = (logcrim-min(logcrim))/(max(logcrim)-min(logcrim))
boston_features["DIS"] = (logdis-min(logdis))/(max(logdis)-min(logdis))

#standardization
boston_features["AGE"] = (age-np.mean(age))/np.sqrt(np.var(age))
boston_features["INDUS"] = (logindus-np.mean(logindus))/np.sqrt(np.var(logindus))
boston_features["LSTAT"] = (loglstat-np.mean(loglstat))/np.sqrt(np.var(loglstat))
boston_features["PTRATIO"] = (logptratio-np.mean(logptratio))/(np.sqrt(np.var(logptratio)))

## Perform stepwise selection

The code for stepwise selection is copied below.

In [2]:
import statsmodels.api as sm

def stepwise_selection(X, y, 
                       initial_list=[], 
                       threshold_in=0.01, 
                       threshold_out = 0.05, 
                       verbose=True):
    """ Perform a forward-backward feature selection 
    based on p-value from statsmodels.api.OLS
    Arguments:
        X - pandas.DataFrame with candidate features
        y - list-like with the target
        initial_list - list of features to start with (column names of X)
        threshold_in - include a feature if its p-value < threshold_in
        threshold_out - exclude a feature if its p-value > threshold_out
        verbose - whether to print the sequence of inclusions and exclusions
    Returns: list of selected features 
    Always set threshold_in < threshold_out to avoid infinite looping.
    See https://en.wikipedia.org/wiki/Stepwise_regression for the details
    """
    included = list(initial_list)
    while True:
        changed=False
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded)
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.idxmin()
            included.append(best_feature)
            changed=True
            if verbose:
                print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        # backward step
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.argmax()
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included

In [3]:
stepwise_selection(boston_features, boston_features["PTRATIO"])

Add  PTRATIO                        with p-value 0.0
Add  CRIM                           with p-value 1.55499e-21
Add  B                              with p-value 1.31214e-48
Drop CRIM                           with p-value 0.260819


/opt/conda/envs/learn-env/lib/python3.6/site-packages/ipykernel_launcher.py:45: FutureWarning: 'argmax' is deprecated, use 'idxmax' instead. The behavior of 'argmax'
will be corrected to return the positional maximum in the future.
Use 'series.values.argmax' to get the position of the maximum now.


Add  TAX_(270, 360]                 with p-value 7.44019e-63
Add  INDUS                          with p-value 3.07734e-60
Add  AGE                            with p-value 2.65605e-31
Add  RAD_(0, 6]                     with p-value 1.06034e-07
Drop AGE                            with p-value 0.153887
Add  LSTAT                          with p-value 1.58033e-36
Add  RAD_(6, 24]                    with p-value 4.66892e-17
Drop B                              with p-value 0.239477
Add  B                              with p-value 3.26927e-06
Drop TAX_(270, 360]                 with p-value 0.623188
Add  TAX_(0, 270]                   with p-value 6.12672e-12
Drop RAD_(0, 6]                     with p-value 0.637636
Add  TAX_(270, 360]                 with p-value 0.000136892
Drop B                              with p-value 0.879559
Add  AGE                            with p-value 7.88406e-20
Drop TAX_(270, 360]                 with p-value 0.189006
Add  CRIM                           with p

['PTRATIO',
 'B',
 'TAX_(270, 360]',
 'CHAS',
 'TAX_(360, 712]',
 'INDUS',
 'AGE',
 'RAD_(0, 6]',
 'RAD_(6, 24]',
 'LSTAT',
 'TAX_(0, 270]']

In [4]:
features = ['PTRATIO',
 'B',
 'TAX_(270, 360]',
 'CHAS',
 'TAX_(360, 712]',
 'INDUS',
 'AGE',
 'RAD_(0, 6]',
 'RAD_(6, 24]',
 'LSTAT',
 'TAX_(0, 270]']

### Build the final model again in Statsmodels

In [12]:
features = [f for f in features if "(" not in f]
formula = "PTRATIO~%s" % "+".join(features[1:])

In [13]:
formula

'PTRATIO~B+CHAS+INDUS+AGE+LSTAT'

In [14]:
from statsmodels.formula.api import ols
model = ols(formula=formula, data=boston_features).fit()

In [15]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                PTRATIO   R-squared:                       0.232
Model:                            OLS   Adj. R-squared:                  0.225
Method:                 Least Squares   F-statistic:                     30.27
Date:                Tue, 07 May 2019   Prob (F-statistic):           6.70e-27
Time:                        17:56:47   Log-Likelihood:                -651.09
No. Observations:                 506   AIC:                             1314.
Df Residuals:                     500   BIC:                             1340.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0028      0.170      0.017      0.987      -0.331       0.336
B              0.0317      0.184      0.172      0.863      -0.330       0.393
CHAS          -0.4529      0.158     -2.864      0.004      -0.764      -0.142
INDUS          0.3480      0.055      6.297      0.000       0.239       0.457
AGE           -0.1254      0.054     -2.314      0.021      -0.232      -0.019
LSTAT          0.2589      0.055      4.710      0.000       0.151       0.367
==============================================================================
Omnibus:                       78.612   Durbin-Watson:                   0.301
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              113.033
Skew:                          -1.072   Prob(JB):                     2.85e-25
Kurtosis:                       3.875   Cond. No.                         9.54
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Where our stepwise procedure mentions that "CHAS" was added with a p-value of 0.00151282, but our statsmodels output returns a p-value of 0.000. What is the intuition behind this?

## Use Feature ranking with recursive feature elimination

Use feature ranking to select the 5 most important features

In [17]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
linreg = LinearRegression()


In [35]:
features

['PTRATIO', 'B', 'CHAS', 'INDUS', 'AGE', 'LSTAT']

In [36]:
linreg = LinearRegression()
rfe = RFE(linreg, n_features_to_select = 2).fit(boston_features[features[1:]], boston_features["PTRATIO"])


In [37]:
rfe.ranking_

array([4, 1, 1, 3, 2])

In [38]:
rfe.support_

array([False,  True,  True, False, False])

Fit the linear regression model again using the 5 columns selected

In [28]:
boston_features.columns[6]

'PTRATIO'

In [39]:
from statsmodels.formula.api import ols
formula = "PTRATIO~%s" % "+".join([features[1], features[2]])
model = ols(formula=formula, data=boston_features).fit()

In [40]:
features

['PTRATIO', 'B', 'CHAS', 'INDUS', 'AGE', 'LSTAT']

In [41]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                PTRATIO   R-squared:                       0.040
Model:                            OLS   Adj. R-squared:                  0.036
Method:                 Least Squares   F-statistic:                     10.44
Date:                Tue, 07 May 2019   Prob (F-statistic):           3.62e-05
Time:                        18:05:39   Log-Likelihood:                -707.69
No. Observations:                 506   AIC:                             1421.
Df Residuals:                     503   BIC:                             1434.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.6637      0.176      3.766      0.000       0.317       1.010
B             -0.7057      0.190     -3.710      0.000      -1.079      -0.332
CHAS          -0.4279      0.172     -2.482      0.013      -0.767      -0.089
==============================================================================
Omnibus:                       68.830   Durbin-Watson:                   0.224
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               94.114
Skew:                          -1.035   Prob(JB):                     3.66e-21
Kurtosis:                       3.422   Cond. No.                         7.98
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Now, predict $\hat y$ using your model. you can use `.predict()` in scikit-learn

In [43]:
y_p = model.predict()

Now, using the formulas of R-squared and adjusted-R-squared below, and your Python/numpy knowledge, compute them and contrast them with the R-squared and adjusted-R-squared in your statsmodels output using stepwise selection. Which of the two models would you prefer?

$SS_{residual} = \sum (y - \hat{y})^2 $

$SS_{total} = \sum (y - \bar{y})^2 $

$R^2 = 1- \dfrac{SS_{residual}}{SS_{total}}$

$R^2_{adj}= 1-(1-R^2)\dfrac{n-1}{n-p-1}$

In [44]:
y = boston_features["PTRATIO"]

In [50]:
s_res = sum((y-y_p)**2)
print(s_res)

485.83647115389005


In [51]:
s_tot = sum((y-np.mean(y)**2))
print(s_tot)

-1.2505552149377763e-12


In [52]:
r2 = 1 - (s_res/s_tot)
print(r2)

388496617622809.25


In [ ]:
1 - ( (1 - r2)

## Level up - Optional

- Perform variable selection using forward selection, using this resource: https://planspace.org/20150423-forward_selection_with_statsmodels/. Note that this time features are added based on the adjusted-R-squared!
- Tweak the code in the `stepwise_selection()`-function written above to just perform forward selection based on the p-value.

## Summary
Great! You now performed your own feature selection methods!